# Purpose

- Converts the geospatial data within `stamdata_2022.xlsx` from `UTM 32N` to `Lat/Lng`.
- Convert the `.gpkg` file to csv.

### Input files

TODO

### Outputs files
- `wind_turbines.csv`
- `wind_parks.json`

In [110]:
import pandas as pd
import geopandas as gpd
import shapely
import utm

In [20]:
try:
    data = pd.read_excel("stamdata_2022.xlsx")
except Exception as e:
    print("Unable to find the offshore wind dataset - please download it into the data folder as shown in the README")
    print(e)

In [60]:
# Clean data and remove rows with error
data = data.dropna(subset=['UTMx', 'UTMy'], axis=0)
data = data[~data.StamdataGSRN.isin([0, 570714700000004766])]

In [70]:
data.head()

,StamdataGSRN,Idriftsdato,Afmeldtdato,InstalleretkW,Rotordiameter,Navhøjde,Fabrikat,Model,Kommune,Placeringstype,...,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
0,570714700000000010,1989-07-14,2005-10-17,95.0,19.0,22.6,Tellus,TE 1995,155 - Dragør,LAND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,570714700000000102,1992-12-14,2009-06-28,300.0,31.0,30.0,BONUS,COMBI,167 - Hvidovre,LAND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,570714700000000133,1993-12-17,2009-10-08,1000.0,50.0,55.0,EK-ElTech,50/1000,167 - Hvidovre,LAND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,570714700000000140,2000-05-17,NaT,660.0,47.0,40.0,Vestas Wind Systems A/S,V 47,167 - Hvidovre,LAND,...,89603.365,78051.121,76711.438,31069.664,74389.216,21846.964,69309.994,0.000,114317.162,113544.282
4,570714700000000157,2000-05-17,NaT,660.0,47.0,40.0,Vestas Wind Systems A/S,V 47,167 - Hvidovre,LAND,...,92542.848,78552.183,74089.644,28853.890,74260.163,18435.310,89814.343,93481.274,177889.977,102833.133


In [68]:
geo_coords = data.apply(lambda row: pd.Series(utm.to_latlon(row.UTMx, row.UTMy, 32, 'N'), index=['lat', 'lng']), axis=1, result_type='expand')

In [72]:
data = pd.concat([data, geo_coords], axis=1)
data.to_csv('wind_turbines.csv')

### Convert `.gpkg` file

# TO REMOVE??

In [142]:
try:
    data = gpd.read_file("ens_havvind.gpkg")
except Exception as e:
    print("Unable to find the offshore wind dataset - please download `ens_havvind.gpkg` into the data folder as shown in the README")
    print(e)

In [143]:
for itt, geom in enumerate(data.geometry):
    print(geom)
    latlng_geom = []
    for g in geom.geoms:
        print(g)
        latlng_g = []
        for point in g.exterior.coords:
            lat, lng = utm.to_latlon(point[0], point[1], 32, "N")
            latlng_g.append((lng, lat))
        latlng_geom.append(latlng_g)
        
    # Create final object
    ob = shapely.geometry.Polygon(latlng_g)
    data.geometry[itt] = ob

MULTIPOLYGON (((740042.425 6178209.959, 740114.736 6178121.848, 740522.38 6177625.132, 740849.485 6177226.553, 741076.6 6176801.652, 741944.575 6175177.788, 742444.908 6173943.374, 742915 6172393.687, 743193.147 6170764.162, 743320.623 6169469.864, 743305.079 6168942.594, 744315.807 6169902.196, 745029.119 6173271.502, 741683.544 6179150.931, 741433.503 6179516.611, 741188.089 6179286.092, 740739.75 6178864.963, 740042.425 6178209.959)))
POLYGON ((740042.425 6178209.959, 740114.736 6178121.848, 740522.38 6177625.132, 740849.485 6177226.553, 741076.6 6176801.652, 741944.575 6175177.788, 742444.908 6173943.374, 742915 6172393.687, 743193.147 6170764.162, 743320.623 6169469.864, 743305.079 6168942.594, 744315.807 6169902.196, 745029.119 6173271.502, 741683.544 6179150.931, 741433.503 6179516.611, 741188.089 6179286.092, 740739.75 6178864.963, 740042.425 6178209.959))
MULTIPOLYGON (((731726 6083457, 730446 6082977, 729915.999999999 6082827, 728105.000000001 6082357, 727295.000000001 608225

In [144]:
data.to_file('wind_parks.geojson', driver="GeoJSON")  